# Using Shotgun metagenomics for disease prediction
Shotgun metagenomic sequencing is a relatively new sequencing approach that allows insight to be gained into community biodiversity and function.<br>
The function of shotgun metagenomic sequencing is to sequence the genomes of untargeted cells in a community in order to elucidate community composition and function.<br>
In this notebook I depart from the discoveries made in the [dataset's creators research paper](https://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1004977), and therefore skip EDA. I wanted to expand on their work and first of all I approach prediction cirrhosis using XGBoost in 'synthetic pure' case of samples with cirrhosis against healthy samples. Creating binary classification problem this way, and comparing it with the result they achieved using RandomForest and SVM. <br>
I'm also checking if the model trained on this dataset would be still able to make prediction if we given a dataset with other diseases mixed in. <br>
And finally I want to check if the model will be able to distinguish cirrhosis from other diseases.

![](https://d3i71xaburhd42.cloudfront.net/fb32dc53b3f5c638d84fb02da7213863a27cee28/2-Figure1-1.png)


A few useful articles:<br>
[An introduction to the analysis of shotgun metagenomic data](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4059276/)<br>
[Shotgun metagenomics, from sampling to analysis](https://pubmed.ncbi.nlm.nih.gov/28898207/)<br>
[16S rRNA Gene Sequencing for Bacterial Identification in the Diagnostic Laboratory: Pluses, Perils, and Pitfalls](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2045242/)<br>
[Then and now: use of 16S rDNA gene sequencing for bacterial identification and discovery of novel bacteria in clinical microbiology laboratories](https://pubmed.ncbi.nlm.nih.gov/18828852/)<br>
[Direct 16S rRNA-seq from bacterial communities: a PCR-independent approach to simultaneously assess microbial diversity and functional activity potential of each taxon](https://www.nature.com/articles/srep32165#abstract)<br>


Loading libraries

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

from xgboost import XGBClassifier
import xgboost as xgb

import sklearn
from sklearn import metrics
from sklearn import preprocessing
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_auc_score, confusion_matrix, precision_recall_curve, auc, roc_curve, recall_score

import seaborn as sns
import scikitplot as skplt
import matplotlib.pyplot as plt


/kaggle/input/human-metagenomics/markers2clades_DB.csv
/kaggle/input/human-metagenomics/abundance.csv
/kaggle/input/human-metagenomics/abundance_stoolsubset.csv
/kaggle/input/metagenomics/markers2clades_DB.txt
/kaggle/input/metagenomics/abundance_stoolsubset.txt
/kaggle/input/metagenomics/abundance.txt
/kaggle/input/metagenomics/marker_presence.txt


In [2]:
# Metaparameters
VERBOSE = 0
FOLDS = 5

show_fold_stats = True
# show_fold_stats = False # set to True if all OOF results wanted

do_plot_ROC = False # set to True to plot ROC when predicting cirrhosis

# test_train_split_SEED = 1970
test_train_split_SEED = 1971

In [3]:
def plot_ROC(fpr, tpr, m_name):
    roc_auc = sklearn.metrics.auc(fpr, tpr)
    plt.figure(figsize=(6, 6))
    lw = 2
    plt.plot(fpr, tpr, color='darkorange',
             lw=lw, label='ROC curve (area = %0.2f)' % roc_auc, alpha=0.5)
    
    plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--', alpha=0.5)
    
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    plt.grid(True)
    plt.xlabel('False Positive Rate', fontsize=16)
    plt.ylabel('True Positive Rate', fontsize=16)
    plt.title('ROC for %s'%m_name, fontsize=20)
    plt.legend(loc="lower right", fontsize=16)
    plt.show()

My initial experiments with full dataset of species abundance features proved that XGB wasn't able to beat the original paper results. So I benefited from their analyses of features importance, and used the reduced dataset for ongoing experimentation. <br>
I still plan on using markers in future experiments.

In [4]:

# pd_abundance = pd.read_csv('/kaggle/input/metagenomics/abundance.txt', sep='\t', header=None, index_col=0, dtype=str)
pd_abundance = pd.read_csv('../input/human-metagenomics/abundance_stoolsubset.csv', dtype=str)
disease = pd_abundance.loc[:,'disease'] 
d_name = pd_abundance.loc[:,'dataset_name'] 
print(disease.value_counts())
print(d_name.value_counts())

# list of diseases we want to analyze and predict
diseases = ['obesity', 'cirrhosis', 't2d', 'cancer']

n                             944
t2d                           223
obesity                       164
ibd_ulcerative_colitis        148
cirrhosis                     118
leaness                        89
stec2-positive                 52
impaired_glucose_tolerance     49
cancer                         48
n_relative                     47
small_adenoma                  26
ibd_crohn_disease              25
 -                             20
large_adenoma                  13
overweight                     10
-                               7
obese                           5
underweight                     1
Name: disease, dtype: int64
Neilsen_genome_assembly              382
t2dmeta_long                         290
Chatelier_gut_obesity                278
Quin_gut_liver_cirrhosis             232
hmp                                  152
WT2D                                 145
Zeller_fecal_colorectal_cancer       134
metahit                              110
t2dmeta_short                   

In [5]:
cols = pd_abundance.columns.tolist()
species = [x for x in cols if x.startswith('k_')]
metadata = [x for x in cols if not x.startswith('k_')]
pd_abundance_conv = pd_abundance.copy()
pd_abundance_conv = pd_abundance_conv[species].astype('float64')
pd_abundance_conv = pd.concat([pd_abundance[metadata], pd_abundance_conv], axis = 1)

# controls/healthy samples from Human Microbiome Project coded 'hmp' and 'hmpii'. 
# 't2d' stands for Type 2 Diabetes. We will combine a few studies into single dataset.
data_sets = {'control':['hmp', 'hmpii'],'t2d':['WT2D','t2dmeta_long','t2dmeta_short'], 'cirrhosis' : ['Quin_gut_liver_cirrhosis'], 
             'cancer' : ['Zeller_fecal_colorectal_cancer'], 'obesity' : ['Chatelier_gut_obesity']}
# combine controls from different studies into one
pd_abundance_conv['disease'] = pd_abundance_conv['disease'].apply(lambda x: 'control' if ((x == 'n') or (x == 'nd') or (x == 'leaness')) else x)
# separate controls and diseases into 2 dataframes
pd_control = pd_abundance_conv.loc[pd_abundance_conv['disease'] == 'control']
pd_disease = pd_abundance_conv.loc[pd_abundance_conv['disease'] != 'control']

# we won't consider diseases from this list
not_disease = [d for d in pd_disease.disease.unique().tolist() if d not in diseases] 
for d in not_disease:
    pd_disease = pd_disease.drop(pd_disease.loc[pd_disease['disease'] == d].index, axis = 0)     

## Defining model

In [6]:
def disease_distinction(d, counter_d):
    print('-' * 80)
    print('Discriminate %s from %s'%(d,counter_d))
    
    skf = StratifiedKFold(n_splits = FOLDS, shuffle = True, random_state = SEED)
    oof_preds = []
    oof_aucs = []
    oof_scores= []
    ds_names = data_sets[d]
    if len(ds_names) == 1:
        pd_cont = pd_control.loc[pd_control['dataset_name'] == ds_names[0]]
        pd_dis = pd_disease.loc[pd_disease['dataset_name'] == ds_names[0]]
    else:
        pd_cont = pd_control.loc[pd_control['dataset_name'] == ds_names[0]]
        pd_dis = pd_disease.loc[(pd_disease['disease'] == d) & (pd_disease['dataset_name'] == ds_names[0])]
        for ds in ds_names[1:]:
            pd_cont = pd.concat([pd_cont, pd_control.loc[pd_control['dataset_name'] == ds]], axis = 0)
            pd_dis = pd.concat([pd_dis, pd_disease.loc[(pd_disease['disease'] == d) & (pd_disease['dataset_name'] == ds)]], axis = 0)
                                 
    #     create dataset with all other diseases, target set to 0
    pd_others = pd_disease.loc[(pd_disease['disease'] == counter_d)]
    target_others = pd_others['disease'].apply(lambda x: 1 if x == d else 0)
    
    pd_train = pd.concat([pd_cont, pd_dis], axis = 0)
    #     adding control data from healthy subject, data by HMP
    pd_train = pd.concat([pd_train, pd_control.loc[pd_control['dataset_name'] == 'hmp']], axis = 0)
    pd_train = pd.concat([pd_train, pd_control.loc[pd_control['dataset_name'] == 'hmpii']], axis = 0)
    target = pd_train['disease']
    #     convert text target into binary
    binary_target = target.apply(lambda x: 1 if x == d else 0)
    # binary_target.value_counts()
    #     use only abundance data for training
    pd_others = pd_others[species] 
    pd_train  = pd_train[species] 

    #     this split provides us with preserved test set
    disease_train, disease_test, disease_y_train, disease_y_test = train_test_split(pd_train, binary_target, test_size = 0.10, 
#                                                                                     stratify = binary_target,
                                                                                    random_state = test_train_split_SEED)   

    #     combining preserved test set with othere diseases samples
    full_test = pd.concat([disease_test, pd_others])
    full_y_test = pd.concat([disease_y_test, target_others])
    
    disease_y_test.value_counts()
    disease_y_train.value_counts()
    preds = np.zeros(disease_y_test.shape[0])
    full_preds = np.zeros(full_y_test.shape[0])

    for fold, (idxT,idxV) in enumerate(skf.split(disease_train, disease_y_train)):

        X_train = disease_train.iloc[idxT]
        X_val = disease_train.iloc[idxV]
        y_train = disease_y_train.iloc[idxT]
        y_val = disease_y_train.iloc[idxV]
    
        XGB_model = XGBClassifier(n_estimators=5000, max_depth=None, 
                            learning_rate=0.05,
                            objective='binary:logistic', 
                            metric='auc',
                            verbosity  = VERBOSE,
                            n_jobs=-1, random_state  = SEED)
        
        if show_fold_stats:
            print('-' * 80)
            print('Fold : %s'%(fold+1))
    
        XGB_model.fit(X_train, y_train,
                        eval_set = [(X_val, y_val)],
                        eval_metric=['logloss'],
                        early_stopping_rounds = 100, verbose = VERBOSE )
            
        XGB_preds = XGB_model.predict_proba(X_val)
        XGB_score = metrics.roc_auc_score(y_val, XGB_preds[:,1])
        XGB_class = XGB_model.predict(X_val)
        
        XGB_test = XGB_model.predict_proba(disease_test)
        XGB_test_score = metrics.roc_auc_score(disease_y_test, XGB_test[:,1])
        XGB_test_class = XGB_model.predict(disease_test)
        
        full_test_preds = XGB_model.predict_proba(full_test)
        full_test_score = metrics.roc_auc_score(full_y_test, full_test_preds[:,1])
        full_test_class = XGB_model.predict(full_test)
               
        f1s = f1_score(y_val, XGB_class)
        recall = metrics.recall_score(y_val, XGB_class)
        precision_score = metrics.precision_score(y_val, XGB_class)
        
        f1_test = f1_score(disease_y_test, XGB_test_class)
        recall_test = metrics.recall_score(disease_y_test, XGB_test_class)
        precision_score_test = metrics.precision_score(disease_y_test, XGB_test_class)
        
        f1_full_test = f1_score(full_y_test, full_test_class)
        recall_full_test = metrics.recall_score(full_y_test, full_test_class)
        precision_full_test = metrics.precision_score(full_y_test, full_test_class)

        if show_fold_stats:        
            print('ROC AUC score for XGBoost model valid set: %.4f'%XGB_score)
            print('F1 score: %0.4f'%f1s)
            print(confusion_matrix(y_val, XGB_class))
    
            print('ROC AUC score for XGBoost model test set: %.4f'%XGB_test_score)
            print('F1 score: %0.4f'%f1_test)
            print(confusion_matrix(disease_y_test, XGB_test_class))
            
            print('ROC AUC score for full test set including other diseases used as false controls: %.4f'%full_test_score)
            print('F1 score: %0.4f'%f1_full_test)
            print(confusion_matrix(full_y_test, full_test_class))
        
        preds += XGB_test[:,1] / FOLDS
        full_preds += full_test_preds[:,1] / FOLDS
        
        fold_score = [XGB_score,f1s,recall,precision_score, XGB_test_score,f1_test,recall_test,precision_score_test]
        oof_scores.append({fold : fold_score})
    
    avg_test_score = metrics.roc_auc_score(disease_y_test, preds)
    avg_class = np.where(preds < 0.5, 0, 1)
    avg_f1_test = f1_score(disease_y_test, avg_class)
    avg_recall_test = metrics.recall_score(disease_y_test, avg_class)
    avg_precision_score_test = metrics.precision_score(disease_y_test, avg_class)

    avg_full_test_score = metrics.roc_auc_score(full_y_test, full_preds)
    avg_class_full = np.where(full_preds < 0.5, 0, 1)
    avg_f1_test_full = f1_score(full_y_test, avg_class_full)
    avg_recall_full_test = metrics.recall_score(full_y_test, avg_class_full)
    avg_precision_full_test = metrics.precision_score(full_y_test, avg_class_full)

    if show_fold_stats:        
        print('-' * 80)
        print('ROC AUC score for %s averaged over %i folds: %.4f'%(d, FOLDS, avg_test_score))
        print('F1 : %.4f, Recall : %.4f , Precision : %.4f'%(avg_f1_test, avg_recall_test, avg_precision_score_test))
        print('Confusion matrix for %s averaged across %i folds '%(d,FOLDS))
        print(confusion_matrix(disease_y_test, avg_class))

    
    print('ROC AUC score for %s against %s averaged over %i folds : %.4f'%(d, counter_d, FOLDS, avg_full_test_score ))
    print('F1 : %.4f, Recall : %.4f , Precision : %.4f'%(avg_f1_test_full, avg_recall_full_test, avg_precision_full_test))
    print('Confusion matrix for %s against %s averaged across %i folds'%(d, counter_d, FOLDS))
    print(confusion_matrix(full_y_test, avg_class_full))
    
    return full_preds, full_y_test, {d : (avg_test_score, avg_f1_test, avg_recall_test, avg_precision_score_test, oof_scores)}


def disease_prediction(d):
    print('-' * 80)
    print('Disease : %s'%d)
    
    skf = StratifiedKFold(n_splits = FOLDS, shuffle = True, random_state = SEED)
    oof_preds = []
    oof_aucs = []
    oof_scores= []
    ds_names = data_sets[d]
    if len(ds_names) == 1:
        pd_cont = pd_control.loc[pd_control['dataset_name'] == ds_names[0]]
        pd_dis = pd_disease.loc[pd_disease['dataset_name'] == ds_names[0]]
    else:
        pd_cont = pd_control.loc[pd_control['dataset_name'] == ds_names[0]]
        pd_dis = pd_disease.loc[(pd_disease['disease'] == d) & (pd_disease['dataset_name'] == ds_names[0])]
        for ds in ds_names[1:]:
            pd_cont = pd.concat([pd_cont, pd_control.loc[pd_control['dataset_name'] == ds]], axis = 0)
            pd_dis = pd.concat([pd_dis, pd_disease.loc[(pd_disease['disease'] == d) & (pd_disease['dataset_name'] == ds)]], axis = 0)
                
    #     create dataset with all other diseases, target set to 0
    pd_others = pd_disease.loc[(pd_disease['disease'] != d)]
    target_others = pd_others['disease'].apply(lambda x: 1 if x == d else 0)
    
    #     combine controls and this particular disease back into train dataset
    pd_train = pd.concat([pd_cont, pd_dis], axis = 0)

    #     adding control data from healthy subject, data by HMP
    pd_train = pd.concat([pd_train, pd_control.loc[pd_control['dataset_name'] == 'hmp']], axis = 0)
    pd_train = pd.concat([pd_train, pd_control.loc[pd_control['dataset_name'] == 'hmpii']], axis = 0)

    target = pd_train['disease']
    #     convert text target into binary
    binary_target = target.apply(lambda x: 1 if x == d else 0)
    # binary_target.value_counts()
    #     use only abundance features data for training
    pd_others = pd_others[species] 
    pd_train  = pd_train[species] 
    #     this split provides us with preserved test set
    disease_train, disease_test, disease_y_train, disease_y_test = train_test_split(pd_train, binary_target, test_size = 0.10, 
#                                                                                     stratify = binary_target,
                                                                                    random_state = test_train_split_SEED)   
        
    #     combining preserved test set with othere diseases samples
    full_test = pd.concat([disease_test, pd_others])
    full_y_test = pd.concat([disease_y_test, target_others])
    
    disease_y_test.value_counts()
    disease_y_train.value_counts()
    preds = np.zeros(disease_y_test.shape[0])
    full_preds = np.zeros(full_y_test.shape[0])

    for fold, (idxT,idxV) in enumerate(skf.split(disease_train, disease_y_train)):

        X_train = disease_train.iloc[idxT]
        X_val = disease_train.iloc[idxV]
        y_train = disease_y_train.iloc[idxT]
        y_val = disease_y_train.iloc[idxV]
    
        XGB_model = XGBClassifier(n_estimators=5000, max_depth=None, 
                            learning_rate=0.05,
                            objective='binary:logistic', 
                            metric='auc',
                            verbosity  = VERBOSE,
                            n_jobs=-1, random_state  = SEED)
        
        if show_fold_stats:
            print('-' * 80)
            print('Fold : %s'%(fold+1))
    
        XGB_model.fit(X_train, y_train,
                        eval_set = [(X_val, y_val)],
                        eval_metric=['logloss'],
                        # eval_metric=['auc','logloss'],
                        early_stopping_rounds = 100, verbose = VERBOSE )
            
        XGB_preds = XGB_model.predict_proba(X_val)
        XGB_score = metrics.roc_auc_score(y_val, XGB_preds[:,1])
        XGB_class = XGB_model.predict(X_val)
        
        XGB_test = XGB_model.predict_proba(disease_test)
        XGB_test_score = metrics.roc_auc_score(disease_y_test, XGB_test[:,1])
        XGB_test_class = XGB_model.predict(disease_test)
        
        full_test_preds = XGB_model.predict_proba(full_test)
        full_test_score = metrics.roc_auc_score(full_y_test, full_test_preds[:,1])
        full_test_class = XGB_model.predict(full_test)
              
        f1s = f1_score(y_val, XGB_class)
        recall = metrics.recall_score(y_val, XGB_class)
        precision_score = metrics.precision_score(y_val, XGB_class)
        
        f1_test = f1_score(disease_y_test, XGB_test_class)
        recall_test = metrics.recall_score(disease_y_test, XGB_test_class)
        precision_score_test = metrics.precision_score(disease_y_test, XGB_test_class)
        
        f1_full_test = f1_score(full_y_test, full_test_class)
        recall_full_test = metrics.recall_score(full_y_test, full_test_class)
        precision_full_test = metrics.precision_score(full_y_test, full_test_class)
        
        if show_fold_stats:        
            print('ROC AUC score for XGBoost model valid set: %.4f'%XGB_score)
            print('F1 score: %0.4f'%f1s)
            print(confusion_matrix(y_val, XGB_class))
    
            print('ROC AUC score for XGBoost model test set: %.4f'%XGB_test_score)
            print('F1 score: %0.4f'%f1_test)
            print(confusion_matrix(disease_y_test, XGB_test_class))
            
            print('ROC AUC score for full test set including other diseases used as false controls: %.4f'%full_test_score)
            print('F1 score: %0.4f'%f1_full_test)
            print(confusion_matrix(full_y_test, full_test_class))

        preds += XGB_test[:,1] / FOLDS
        full_preds += full_test_preds[:,1] / FOLDS
        
        fold_score = [XGB_score,f1s,recall,precision_score, XGB_test_score,f1_test,recall_test,precision_score_test]
        oof_scores.append({fold : fold_score})
    
    avg_test_score = metrics.roc_auc_score(disease_y_test, preds)
    avg_class = np.where(preds < 0.5, 0, 1)
    avg_f1_test = f1_score(disease_y_test, avg_class)
    avg_recall_test = metrics.recall_score(disease_y_test, avg_class)
    avg_precision_score_test = metrics.precision_score(disease_y_test, avg_class)

    avg_full_test_score = metrics.roc_auc_score(full_y_test, full_preds)
    avg_class_full = np.where(full_preds < 0.5, 0, 1)
    avg_f1_test_full = f1_score(full_y_test, avg_class_full)
    avg_recall_full_test = metrics.recall_score(full_y_test, avg_class_full)
    avg_precision_full_test = metrics.precision_score(full_y_test, avg_class_full)

    print('-' * 80)
    print('Averaged over %i folds ROC AUC score for %s: %.4f'%(FOLDS,d,avg_test_score))
    print('F1 : %.4f, Recall : %.4f , Precision : %.4f'%(avg_f1_test, avg_recall_test, avg_precision_score_test))
    print('Confusion matrix for %s averaged across %i folds '%(d,FOLDS))
    print(confusion_matrix(disease_y_test, avg_class))

    
    print('Averaged over %i folds ROC AUC score for %s against full set of other diseases: %.4f'%(FOLDS,d,avg_full_test_score))
    print('F1 : %.4f, Recall : %.4f , Precision : %.4f'%(avg_f1_test_full, avg_recall_full_test, avg_precision_full_test))
    print('Confusion matrix for %s averaged across %i folds for full set'%(d,FOLDS))
    print(confusion_matrix(full_y_test, avg_class_full))

    if do_plot_ROC:
        (fpr, tpr, thresholds) = metrics.roc_curve(disease_y_test, preds)
        plot_ROC(fpr, tpr, d)

    return preds, disease_y_test, full_preds, full_y_test

## Predictions
We'll use multiple random seeds to have variations with model initialization, and data splits. We will check if ensembling these multiple predictions will give us a better prediction, of if there will be a better seed.

In [7]:
SEEDs = [42,1970,1971,2020,2021]        
d = 'cirrhosis'
d_preds = []
df_preds = []
dc_preds = []
for SEED in SEEDs:
    print('=' * 80)
    print('Seed : %i'%SEED)
    d_pred, y_true, full_preds, full_y_test = disease_prediction(d) 
    d_preds.append(d_pred)
    df_preds.append(full_preds)

Seed : 42
--------------------------------------------------------------------------------
Disease : cirrhosis
--------------------------------------------------------------------------------
Fold : 1
ROC AUC score for XGBoost model valid set: 0.9946
F1 score: 0.9444
[[58  0]
 [ 2 17]]
ROC AUC score for XGBoost model test set: 1.0000
F1 score: 1.0000
[[23  0]
 [ 0 20]]
ROC AUC score for full test set including other diseases used as false controls: 0.9910
F1 score: 0.5882
[[430  28]
 [  0  20]]
--------------------------------------------------------------------------------
Fold : 2
ROC AUC score for XGBoost model valid set: 0.8707
F1 score: 0.6486
[[52  6]
 [ 7 12]]
ROC AUC score for XGBoost model test set: 0.9913
F1 score: 0.9500
[[22  1]
 [ 1 19]]
ROC AUC score for full test set including other diseases used as false controls: 0.9727
F1 score: 0.4810
[[418  40]
 [  1  19]]
--------------------------------------------------------------------------------
Fold : 3
ROC AUC score for XGB

So the model is able to perfectly predict cirrhosis in 'sysnthetic' case when it has to distinguish between healthy samples and samples with the disease. However it gets really confused when we mix 3 unknown diseases to the controls.<br>
The best case scenario we get is with seeds 1971 and 2021 giving us F1 = 0.6129 and Precision = 0.4524.<br>
<br>


In [8]:
print('Averaged predictions across %i seeds'%len(SEEDs))
avg_pred = np.array(d_preds).mean(axis= 0)
avg_class = np.where(avg_pred < 0.65, 0, 1)
print('Averaged over %i seeds ROC AUC score for %s : %.4f'%(len(SEEDs),d, metrics.roc_auc_score(y_true, avg_pred)))
print('F1 score : %.4f'%f1_score(y_true, avg_class))
print('Confusion matrix for %s'%d)
print(confusion_matrix(y_true, avg_class))

avg_pred = np.array(df_preds).mean(axis= 0)
avg_class = np.where(avg_pred < 0.5, 0, 1)
print('Averaged over %i seeds ROC AUC score for %s against full set of other diseases: %.4f'%(len(SEEDs),d, metrics.roc_auc_score(full_y_test, avg_pred)))
print('Confusion matrix for %s against controls and other diseases combined'%d)
print(confusion_matrix(full_y_test, avg_class))
print('F1 score : %.4f'%f1_score(full_y_test, avg_class))
print('Recall : %.4f'%metrics.recall_score(full_y_test, avg_class))
print('Precision : %.4f'%metrics.precision_score(full_y_test, avg_class))

Averaged predictions across 5 seeds
Averaged over 5 seeds ROC AUC score for cirrhosis : 1.0000
F1 score : 0.9744
Confusion matrix for cirrhosis
[[23  0]
 [ 1 19]]
Averaged over 5 seeds ROC AUC score for cirrhosis against full set of other diseases: 0.9902
Confusion matrix for cirrhosis against controls and other diseases combined
[[433  25]
 [  1  19]]
F1 score : 0.5938
Recall : 0.9500
Precision : 0.4318


Averaging results across all variants does not improve prediction in this case, model with the seed 2021 is still the best. <br>
We need significantly less correlated models in the ensemble for it to enhance the prediction.<br>
<br>
However if we adjust threshold of class separation to 0.88 then we get much better F1 and Precision by virtually eliminating false positives (and increasing number of false negatives at the same time, with Recall worsening as well). So as usual it comes to the decision of what is more important - Precision or Recall.

In [9]:
avg_class = np.where(avg_pred < 0.88, 0, 1)
print('Averaged over %i seeds ROC AUC score for %s against full set of other diseases: %.4f'%(len(SEEDs),d, metrics.roc_auc_score(full_y_test, avg_pred)))
print('Confusion matrix for %s against controls and other diseases combined'%d)
print(confusion_matrix(full_y_test, avg_class))
print('F1 score : %.4f'%f1_score(full_y_test, avg_class))
print('Recall : %.4f'%metrics.recall_score(full_y_test, avg_class))
print('Precision : %.4f'%metrics.precision_score(full_y_test, avg_class))

Averaged over 5 seeds ROC AUC score for cirrhosis against full set of other diseases: 0.9902
Confusion matrix for cirrhosis against controls and other diseases combined
[[456   2]
 [  5  15]]
F1 score : 0.8108
Recall : 0.7500
Precision : 0.8824


## Discriminate cirrhosis from other diseases in pairs
As it was discovered in the original paper diabetes is the most difficult to predict based off the abundance features, and I suspect that is what prohibiting the model to discriminate cirrhosis from other samples, and generates false positives. And we are going to check it in the next step.

In [10]:
dc_preds = []
SEED = 1970
print('=' * 80)
print('Seed : %i'%SEED)

for counter in diseases:
    if counter != d:
        dc_pred, y_preds, _ = disease_distinction(d, counter)

Seed : 1970
--------------------------------------------------------------------------------
Discriminate cirrhosis from obesity
--------------------------------------------------------------------------------
Fold : 1
ROC AUC score for XGBoost model valid set: 0.9936
F1 score: 0.9444
[[58  0]
 [ 2 17]]
ROC AUC score for XGBoost model test set: 1.0000
F1 score: 1.0000
[[23  0]
 [ 0 20]]
ROC AUC score for full test set including other diseases used as false controls: 1.0000
F1 score: 0.9756
[[186   1]
 [  0  20]]
--------------------------------------------------------------------------------
Fold : 2
ROC AUC score for XGBoost model valid set: 0.9537
F1 score: 0.7568
[[54  4]
 [ 5 14]]
ROC AUC score for XGBoost model test set: 0.9870
F1 score: 0.9474
[[23  0]
 [ 2 18]]
ROC AUC score for full test set including other diseases used as false controls: 0.9880
F1 score: 0.8182
[[181   6]
 [  2  18]]
--------------------------------------------------------------------------------
Fold : 3
ROC

As we see the model is capable to distinguish cirrhosis from obesity and cancer with high precision. However diabetes leads to a significant number of false positives, lowering precision to about 50%.<br>
We will need to work on feature engineering to improve diabetes prediction. There is hope in usage of markers instead of/in conjunction with abundance features will help, as it's been shown in the original paper.<br>
And of course there is room for improvement in the XGB model's hyper-parameters tuning, as I've used it with basic default parameters only yet.

# Conclusion
Using XGBoost defintely provides significant boost in precision of predicting cirrhosis in comparison with RF results published in the paper. 